# Banco de Dados - A2 

## I - Carregando os dados capturados

Capturamos 3 arquivos geojson que serão utilizados para a nossa tarefa:

+ [Arquivo do perímetro do estado do Piauí](https://github.com/tbrugz/geodata-br)
+ [Arquivo com municípios atingidos por seca em 2015](https://dados.gov.br/dataset/municipios-atingidos-por-eventos-de-seca-em-2015)
+ [Arquivo com o percentual de moradores em domicílios particulares com 1 a 2 salários](https://dados.gov.br/dataset/cgeo_vw_perc_morad_rend_1_a_2)

In [1]:
import json #carregaremos os geojsons em dicionários

#arquivo geojson piaui

f = open('piaui.geojson')
data1 = json.load(f)
f.close()

#arquivo geojson secas

f = open('seca2015.geojson')
data2 = json.load(f)
f.close()

#arquivo geojson renda

f = open('rend_1_2.geojson')
data3 = json.load(f)
f.close()

## II - Criando o banco de dados

Vamos criar um banco de dados NoSQL no Mongodb e armazenar os dados nele

In [3]:
import datetime
import pprint

import pymongo
from pymongo import MongoClient

In [4]:
client = MongoClient('localhost', 27017)

### Criação do banco e das coleções

In [6]:
#criando o banco

db = client.a2_database

#criando as coleções

collection1 = db.piaui_data 
collection2 = db.secas_data
collection3 = db.renda_data


A seguir, iremos inserir os documentos dos arquivos que capturamos nas coleções criadas, como sugerido pelos nomes das mesmas.

### Armazenando os documentos no banco de dados

In [5]:
#a série de documentos de cada geojson está na chave 'feature' de cada dict que carregamos anteriormente

piaui_features = data1['features']
secas_features = data2['features']
renda_features = data3['features']

#agora vamos adicionar os documentos às coleções e salvar os ids em 3 variáveis, como vimos em aula

results1 = collection1.insert_many(piaui_features)
results2 = collection2.insert_many(secas_features)
results3 = collection3.insert_many(renda_features)


Verificando se as coleções e o banco já aparecem, para ter certeza de que os documentos foram adicionados

In [7]:
print(client.list_database_names())
print(db.list_collection_names())

['a2_database', 'admin', 'config', 'local', 'test_database']
['secas_data', 'renda_data', 'piaui_data']



Só pra ter **realmente** certeza que adicionamos corretamente, vamos fazer uma querry procurando nossa cidade na coleção com os municípios do Piauí

In [8]:
collection1.find_one({'properties.name': 'Cocal dos Alves'})

{'_id': ObjectId('61b57bdca9ea537ddb91589f'),
 'type': 'Feature',
 'properties': {'id': '2202729',
  'name': 'Cocal dos Alves',
  'description': 'Cocal dos Alves'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-41.3951733336, -3.5471106105],
    [-41.3671198987, -3.5615461774],
    [-41.3708712137, -3.5674193404],
    [-41.3702509211, -3.5744918938],
    [-41.3616333511, -3.5828053967],
    [-41.3528629563, -3.5852117234],
    [-41.3445294278, -3.588364419],
    [-41.3309408875, -3.6243095073],
    [-41.338722564, -3.6700448706],
    [-41.3433179904, -3.6755980044],
    [-41.3413730721, -3.680397353],
    [-41.3261682973, -3.6869442306],
    [-41.3072902171, -3.6844280624],
    [-41.2904264133, -3.6893136429],
    [-41.2789599905, -3.7003719703],
    [-41.2550582488, -3.7028847004],
    [-41.2389650459, -3.7122698754],
    [-41.2390555161, -3.7203740815],
    [-41.2515145908, -3.7229642052],
    [-41.256988741, -3.7295248631],
    [-41.2608087009, -3.7571667961],
    [-41.27821

## III - Análise utilizando [geopandas](https://geopandas.org/en/stable/)

Agora iremos fazer uma análise dos dados utilizando geopandas.

Para isso, transformaremos as coleções em GeoDataFrames do geopandas:

In [1]:
#Importando bibliotecas necessárias para a análise (vimos na enciclopédia de python do professor)
#https://github.com/rsouza/FGV_Intro_DS/blob/master/notebooks/Tasks_Mapping_Geopandas.ipynb

import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import descartes
import rtree 

import geopandas as gpd
from geopandas.tools import geocode

from shapely.geometry import LineString, MultiPolygon

import folium
from folium import Choropleth, Circle, Marker, GeoJson
from folium.plugins import HeatMap, MarkerCluster

from IPython.display import Image, HTML, IFrame, FileLink, FileLinks 
from IPython.core.display import display

### Obtendo GeoDataFrames das coleções que criamos anteriormente

In [9]:
#Agora vamos transformar as nossas coleções em geodataframes para que possamos fazer as análises

#df limites municípios do piauí
df_piaui = gpd.GeoDataFrame(list(collection1.find({})))

#df municípios que sofreram com seca em 2015
df_secas = gpd.GeoDataFrame(list(collection2.find({})))

#df qntd pessoas com 1 a 2 rendas que moram em domicílios particulares
df_renda = gpd.GeoDataFrame(list(collection3.find({})))

Dando uma olhadinha:

In [15]:
print(type(df_piaui))
df_piaui.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,_id,type,properties,geometry
0,61b57bdca9ea537ddb915865,Feature,"{'id': '2200053', 'name': 'Acauã', 'descriptio...","{'type': 'Polygon', 'coordinates': [[[-40.9597..."
1,61b57bdca9ea537ddb915866,Feature,"{'id': '2200103', 'name': 'Agricolândia', 'des...","{'type': 'Polygon', 'coordinates': [[[-42.6712..."
2,61b57bdca9ea537ddb915867,Feature,"{'id': '2200202', 'name': 'Água Branca', 'desc...","{'type': 'Polygon', 'coordinates': [[[-42.6618..."
3,61b57bdca9ea537ddb915868,Feature,"{'id': '2200251', 'name': 'Alagoinha do Piauí'...","{'type': 'Polygon', 'coordinates': [[[-40.8840..."
4,61b57bdca9ea537ddb915869,Feature,"{'id': '2200277', 'name': 'Alegrete do Piauí',...","{'type': 'Polygon', 'coordinates': [[[-40.7888..."
